In [ ]:
import pandas as pd

In [ ]:
ar = pd.read_csv("/kaggle/input/new-weather-extreme/Into-the-weather-main/data/Daily_ARMSTRONG (AUT)_3987.csv")
cape = pd.read_csv("/kaggle/input/new-weather-extreme/Into-the-weather-main/data/Daily_CAPE DYER_1739.csv")
cold = pd.read_csv("/kaggle/input/new-weather-extreme/Into-the-weather-main/data/Daily_CAPE PARRY A_1633.csv")
comox = pd.read_csv("/kaggle/input/new-weather-extreme/Into-the-weather-main/data/Daily_COLD LAKE A_2832.csv")
gr = pd.read_csv("/kaggle/input/new-weather-extreme/Into-the-weather-main/data/Daily_COMOX A_155.csv")
ha =pd.read_csv("/kaggle/input/new-weather-extreme/Into-the-weather-main/data/Daily_GREENWOOD A_6354.csv")
ho = pd.read_csv("/kaggle/input/new-weather-extreme/Into-the-weather-main/data/Daily_HAINES JUNCTION_1556.csv")
pi = pd.read_csv("/kaggle/input/new-weather-extreme/Into-the-weather-main/data/Daily_HOPEDALE (AUT)_6781.csv")
tr = pd.read_csv("/kaggle/input/new-weather-extreme/Into-the-weather-main/data/Daily_PILOT MOUND (AUT)_3649.csv")
t = pd.read_csv("/kaggle/input/new-weather-extreme/Into-the-weather-main/data/Daily_TRENTON A_5126.csv")


r = pd.concat([ar, cape], axis=0)
r1 = pd.concat([r, cold], axis=0)
r2 = pd.concat([r1, comox], axis=0)
r3 = pd.concat([r2, gr], axis=0)
r4 = pd.concat([r3, ha], axis=0)
r5 = pd.concat([r4, ho], axis=0)
r6 = pd.concat([r5, pi], axis=0)
r7 = pd.concat([r6, tr], axis=0)
data = pd.concat([r7, t], axis=0)




In [ ]:
data.columns

In [ ]:
data = data[["Station Name","Year","Month","Day","Mean Temp (°C)","Heat Deg Days (°C)","Cool Deg Days (°C)","Total Rain (mm)","Total Snow (cm)","Total Precip (mm)"]]

In [ ]:
data['Station Name'].value_counts()

In [ ]:
data = data[data['Station Name'] == 'CAPE DYER']

data

In [ ]:
data.isnull().sum()

In [ ]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
data['Station Name']= label_encoder.fit_transform(data['Station Name']) 


In [ ]:
data.dtypes

In [ ]:
data.interpolate(limit_direction='both')

In [ ]:
data['Total Rain (mm)'].fillna(int(data['Total Rain (mm)'].mean()), inplace=True) 
data['Total Snow (cm)'].fillna(int(data['Total Snow (cm)'].mean()), inplace=True) 
data['Total Precip (mm)'].fillna(int(data['Total Precip (mm)'].mean()), inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data.head()

In [ ]:
data.rename(columns={'Mean Temp (°C)': 'Temperature', 'Total Rain (mm)': 'Rainfall','Total Snow (cm)':'Snowfall','Total Precip (mm)':'Precipitation'}, inplace=True)


In [ ]:
data.head()

In [ ]:
data = data[["Station Name","Year","Month","Day","Temperature","Rainfall","Snowfall","Cool Deg Days (°C)","Precipitation"]]

In [ ]:
data.head()

In [ ]:
data['Extreme_Weather_Condition'] = 'Unknown'

for index, row in data.iterrows():
    if row['Temperature'] <= -30:
        data.at[index, 'Extreme_Weather_Condition'] = 'Extreme Cold'
    elif row['Temperature'] <= 0 and row['Precipitation'] > 0:
        data.at[index, 'Extreme_Weather_Condition'] = 'Freezing Rain'
    elif row['Temperature'] >= 35:
        data.at[index, 'Extreme_Weather_Condition'] = 'Heat Warning'
    elif row['Precipitation'] >= 30:
        data.at[index, 'Extreme_Weather_Condition'] = 'Heavy Rainfall'
    elif row['Snowfall'] >= 10:
        data.at[index, 'Extreme_Weather_Condition'] = 'Heavy Snowfall'
    elif row['Rainfall'] >= 10 or row['Snowfall'] >= 10 or row['Cool Deg Days (°C)'] >= 40:
        data.at[index, 'Extreme_Weather_Condition'] = 'Snow Storm'
    else:
        data.at[index, 'Extreme_Weather_Condition'] = 'Normal'

data

In [ ]:
data['Extreme_Weather_Condition'].value_counts()

In [ ]:
data['Extreme_Weather_Condition']= label_encoder.fit_transform(data['Extreme_Weather_Condition']) 

In [ ]:
data.head()

In [ ]:
import pandas as pd
# Combine 'Year', 'Month', and 'Day' columns into a new 'Date' column
data['Date'] = pd.to_datetime(data[['Year', 'Month', 'Day']])

data = data.sort_values(by='Date', ascending=True)
data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout


# Prepare data for LSTM
X = data[['Year', 'Month', 'Day', 'Temperature', 'Rainfall', 'Snowfall', 'Cool Deg Days (°C)', 'Precipitation']].values
y = data['Extreme_Weather_Condition'].values


# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25,shuffle = False)

# Reshape data for LSTM input (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.3))  # Reduced dropout for better information flow
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=32))  # Removed return_sequences=True for final layer
model.add(Dropout(0.3))
model.add(Dense(units=7, activation='softmax'))
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

predict = model.predict(X_test)

In [ ]:
# Function to forecast extreme weather condition for multiple future dates
def forecast_multiple_days(model, scaler, initial_date, num_days):
    # Create lists to store forecasted dates and extreme weather conditions
    forecast_dates = []
    forecast_weather = []

    # Start forecasting from the initial_date for the next 'num_days'
    current_date = initial_date
    
    for day in range(num_days):
        # Prepare the input data for the current date
        future_data = [current_date.year, current_date.month, current_date.day, 0, 0, 0, 0, 0]  # Replace with appropriate values
        
        # Scale the future date data using the same scaler used for training data
        future_data_scaled = scaler.transform([future_data])
        
        # Reshape the data for LSTM input (samples, timesteps, features)
        future_data_reshaped = np.reshape(future_data_scaled, (1, 1, len(future_data)))
        
        # Predict the extreme weather condition for the future date
        prediction = model.predict(future_data_reshaped)
        predicted_label = np.argmax(prediction)
        
        # Store the forecasted date and extreme weather condition
        forecast_dates.append(current_date)
        forecast_weather.append(predicted_label)
        
        # Update current_date to the next day for the next iteration
        current_date += pd.Timedelta(days=1)
    
    # Create a DataFrame from the forecasted data
    forecast_data = pd.DataFrame({'Date': forecast_dates, 'Forecasted_Weather': forecast_weather})
    
    return forecast_data

# Example: Forecast the next 60 days starting from a specific date
start_date = pd.to_datetime('2023-12-15')  # Replace with your desired start date
forecast_result = forecast_multiple_days(model, scaler, start_date, num_days=25)
print(forecast_result)


In [ ]:
# Define a function to map numerical predictions to labels/categories
def map_to_labels(predictions):
    # Define your mapping based on the numerical values predicted by the model
    # Replace this mapping with your actual mapping logic based on your model's output
    labels = ['Extreme Cold', 'Freezing Rain', 'Heat Warning', 'Rainfall', 'Snowfall', 'Snow Storm', 'Normal']
    return [labels[pred] for pred in predictions]

# Example usage:
predicted_weather_labels = map_to_labels(forecast_result['Forecasted_Weather'])

# Combine the forecasted dates with the corresponding weather condition labels
forecast_result['Forecasted_Weather_Labels'] = predicted_weather_labels

# Display or use the DataFrame containing forecasted dates and labels for the next 60 days
print(forecast_result)


In [ ]:
#Saving Forecasted File to csv
forecast_result.to_csv('ARMSTRONG BC.csv')